In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df_specs = pd.read_csv('Speech-dataset-generation/new_method_test/rvc_audios/data.csv')
df_specs = df_specs.rename(columns={'input': 'audio', 'target': 'sentence'})
df_specs['sentence'] = df_specs['sentence'].apply(lambda x: x.capitalize())

df_specs = df_specs.sample(frac=1, random_state=42).reset_index(drop=True)
df_specs

,audio,sentence
0,/home/g7/Desktop/names-dataset/Speech-dataset-...,Bided at the blue boar inn until the morrow no...
1,/home/g7/Desktop/names-dataset/Speech-dataset-...,I was to stop at niagara falls buffalo and roc...
2,/home/g7/Desktop/names-dataset/Speech-dataset-...,Live as you like best and your character will ...
3,/home/g7/Desktop/names-dataset/Speech-dataset-...,And he was grim when i told him of the invisib...
4,/home/g7/Desktop/names-dataset/Speech-dataset-...,But it did not strike iro merrick as in any wa...
...,...,...
3953,/home/g7/Desktop/names-dataset/Speech-dataset-...,Patience colored up and trembled when margarit...
3954,/home/g7/Desktop/names-dataset/Speech-dataset-...,He was too young then to have opinions of his ...
3955,/home/g7/Desktop/names-dataset/Speech-dataset-...,Doctor robin went back into the room and bange...
3956,/home/g7/Desktop/names-dataset/Speech-dataset-...,And could cover four hundred fifty mégane with...


In [3]:
df_train, df_test = train_test_split(df_specs, test_size=0.2, random_state=42)

In [4]:
df_test

,audio,sentence
149,/home/g7/Desktop/names-dataset/Speech-dataset-...,From that distance the boats drawn upon the sh...
1025,/home/g7/Desktop/names-dataset/Speech-dataset-...,Justus half concealed himself behind one of hi...
2153,/home/g7/Desktop/names-dataset/Speech-dataset-...,There's a fly way over there beyond that old a...
720,/home/g7/Desktop/names-dataset/Speech-dataset-...,My tongue was thick but it moved yes i was soo...
325,/home/g7/Desktop/names-dataset/Speech-dataset-...,Oh won't it it's accepted already said borg br...
...,...,...
1226,/home/g7/Desktop/names-dataset/Speech-dataset-...,Passengers standing at their cabin doors i sho...
736,/home/g7/Desktop/names-dataset/Speech-dataset-...,Has been established at berlin unfortunately s...
3290,/home/g7/Desktop/names-dataset/Speech-dataset-...,That uncle cipriano at once decided must repre...
927,/home/g7/Desktop/names-dataset/Speech-dataset-...,Introducing him to uncle i̇slam and her cousin...


In [5]:
df_train

,audio,sentence
2534,/home/g7/Desktop/names-dataset/Speech-dataset-...,No pencroft replied elvio harding no he is not...
1613,/home/g7/Desktop/names-dataset/Speech-dataset-...,Yes i went into dry aureole in the o p room an...
3537,/home/g7/Desktop/names-dataset/Speech-dataset-...,I suppose the most rudimentary form of his spe...
1117,/home/g7/Desktop/names-dataset/Speech-dataset-...,There is nothing weak or girlish about him in ...
3192,/home/g7/Desktop/names-dataset/Speech-dataset-...,His eyes were very big and they were set so fa...
...,...,...
1130,/home/g7/Desktop/names-dataset/Speech-dataset-...,Gladly will i do so answered perceval so they ...
1294,/home/g7/Desktop/names-dataset/Speech-dataset-...,The true noble is he who smacks of the people ...
860,/home/g7/Desktop/names-dataset/Speech-dataset-...,Which she had not learnt in the day how shall ...
3507,/home/g7/Desktop/names-dataset/Speech-dataset-...,As far as eastern point lighthouse in gloucest...


In [6]:
from datasets import Dataset, DatasetDict, Audio

In [7]:
dataset = DatasetDict()
dataset["train"] = Dataset.from_pandas(df_train).cast_column("audio", Audio())
dataset["test"] = Dataset.from_pandas(df_test).cast_column("audio", Audio())

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence', '__index_level_0__'],
        num_rows: 3166
    })
    test: Dataset({
        features: ['audio', 'sentence', '__index_level_0__'],
        num_rows: 792
    })
})

In [9]:
dataset = dataset.remove_columns(["__index_level_0__"])
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3166
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 792
    })
})

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [12]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
input_str = dataset["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


2024-10-06 03:48:33.699975: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 03:48:33.810278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 03:48:33.810316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 03:48:33.826923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 03:48:33.863271: I tensorflow/core/platform/cpu_feature_guar

Input:                 No pencroft replied elvio harding no he is not dead his pulse still beats he has even uttered a moan but for your boy's sake calm yourself we have need of all our self possession do not make us lose it my friend
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>No pencroft replied elvio harding no he is not dead his pulse still beats he has even uttered a moan but for your boy's sake calm yourself we have need of all our self possession do not make us lose it my friend<|endoftext|>
Decoded w/out special: No pencroft replied elvio harding no he is not dead his pulse still beats he has even uttered a moan but for your boy's sake calm yourself we have need of all our self possession do not make us lose it my friend
Are equal:             True


In [14]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [16]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    
    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [17]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=1)

Map:   0%|          | 0/3166 [00:00<?, ? examples/s]

Map:   0%|          | 0/792 [00:00<?, ? examples/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3166
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 792
    })
})

In [19]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [20]:
model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [23]:
import evaluate

metric = evaluate.load("wer")


In [24]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER-uncased")
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER-uncased")
nlp = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Handle padding masking
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels from their token IDs
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    pred_entities = []
    for result in pred_str:
        result = nlp(result)
        doc_entities = []
        for entity in result:
            # Normalize B-PER and I-PER to PER and include only if PER
            if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
                doc_entities.append((entity['word'], 'PER'))
        # if doc_entities:  # Only append if there are PER entities in the document
        pred_entities.append(len(doc_entities))
        # print("pred_entity: ", pred_entities)

    true_entities = []
    for result in label_str:
        result = nlp(result)
        doc_entities = []
        for entity in result:
            # Normalize B-PER and I-PER to PER and include only if PER
            if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
                doc_entities.append((entity['word'], 'PER'))
        # if doc_entities:  # Only append if there are PER entities in the document
        true_entities.append(len(doc_entities))
        # print("true_entity: ", true_entities)

    ratio = 0
    count = 0
    for i in range(len(pred_entities)):
        if true_entities[i]:
            ratio += 100 * (pred_entities[i] / true_entities[i])
            count += 1
    ratio /= count

    # Calculate WER if needed
    print("computing wer metric")
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"ner percent": ratio, "wer": wer}



In [29]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./names-whisper-en-spectrogram-new-method",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    push_to_hub_model_id="names-whisper-en-spectrogram-new-method",
)


/home/g7/anaconda3/envs/tort/lib/python3.11/site-packages/transformers/training_args.py:1929: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case shahd237/names-whisper-en-spectrogram-new-method).
  warnings.warn(


In [30]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs


In [31]:
trainer.train()

  0%|          | 0/5000 [00:00<?, ?it/s]

/home/g7/anaconda3/envs/tort/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.783, 'grad_norm': 8.736157417297363, 'learning_rate': 4.800000000000001e-07, 'epoch': 0.13}
{'loss': 0.7092, 'grad_norm': 5.784898281097412, 'learning_rate': 9.800000000000001e-07, 'epoch': 0.25}
{'loss': 0.5945, 'grad_norm': 4.9772539138793945, 'learning_rate': 1.48e-06, 'epoch': 0.38}
{'loss': 0.472, 'grad_norm': 4.873668193817139, 'learning_rate': 1.98e-06, 'epoch': 0.51}
{'loss': 0.3266, 'grad_norm': 4.532548427581787, 'learning_rate': 2.4800000000000004e-06, 'epoch': 0.63}
{'loss': 0.2569, 'grad_norm': 3.739192485809326, 'learning_rate': 2.9800000000000003e-06, 'epoch': 0.76}
{'loss': 0.2279, 'grad_norm': 2.9503836631774902, 'learning_rate': 3.48e-06, 'epoch': 0.88}
{'loss': 0.1995, 'grad_norm': 3.121004104614258, 'learning_rate': 3.980000000000001e-06, 'epoch': 1.01}
{'loss': 0.1475, 'grad_norm': 3.479278564453125, 'learning_rate': 4.48e-06, 'epoch': 1.14}
{'loss': 0.1206, 'grad_norm': 3.12341046333313, 'learning_rate': 4.980000000000001e-06, 'epoch': 1.26}
{'loss': 0.

  0%|          | 0/99 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


computing wer metric
{'eval_loss': 0.03738126903772354, 'eval_ner percent': 98.78378378378379, 'eval_wer': 0.8406725380304243, 'eval_runtime': 170.0335, 'eval_samples_per_second': 4.658, 'eval_steps_per_second': 0.582, 'epoch': 25.25}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 7448.7999, 'train_samples_per_second': 10.74, 'train_steps_per_second': 0.671, 'train_loss': 0.023877397344773635, 'epoch': 25.25}


TrainOutput(global_step=5000, training_loss=0.023877397344773635, metrics={'train_runtime': 7448.7999, 'train_samples_per_second': 10.74, 'train_steps_per_second': 0.671, 'total_flos': 2.3072402755584e+19, 'train_loss': 0.023877397344773635, 'epoch': 25.252525252525253})

In [32]:
kwargs = {
    "dataset_tags": "Libri",
    "dataset": "LibriSpeech",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: test",
    "language": "en",
    "model_name": "names Whisper small",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}


In [33]:
trainer.save_model() 
trainer.push_to_hub()
tokenizer.push_to_hub("names-whisper-en-spectrogram-new-method") 


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfi

README.md:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shahd237/names-whisper-en-spectrogram-new-method/commit/8cb5e30353bd2a9a491163fb4650f515239aa88d', commit_message='Upload tokenizer', commit_description='', oid='8cb5e30353bd2a9a491163fb4650f515239aa88d', pr_url=None, pr_revision=None, pr_num=None)

# Testing on random Librispeech

In [1]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
finetuned_model = WhisperForConditionalGeneration.from_pretrained("shahd237/names-whisper-en-spectrogram-new-method")
finetuned_processor = WhisperProcessor.from_pretrained("shahd237/names-whisper-en-spectrogram-new-method")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import librosa
import numpy as np
import whisper
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, Audio

In [3]:
#load data from random_1000_from_librispeech.csv
df_specs = pd.read_csv('random_1000_from_librispeech.csv')
df_specs

,Unnamed: 0,audio,sentence
0,0,/home/g7/Desktop/LibriSpeech/train-clean-100/5...,Olenin was as happy as a boy of twelve tie it ...
1,1,/home/g7/Desktop/LibriSpeech/train-clean-100/3...,I could just make out that he had a book as we...
2,2,/home/g7/Desktop/LibriSpeech/train-clean-100/2...,And what a centrepiece it was it required the ...
3,3,/home/g7/Desktop/LibriSpeech/train-clean-100/6...,The iron was rusty the leather torn the wood w...
4,4,/home/g7/Desktop/LibriSpeech/train-clean-100/7...,Will satisfy my everlasting hatred my courage ...
...,...,...,...
995,995,/home/g7/Desktop/LibriSpeech/train-clean-100/7...,The piano bard the piano rhapsodist the piano ...
996,996,/home/g7/Desktop/LibriSpeech/train-clean-100/3...,Then another and a different horror fell to my...
997,997,/home/g7/Desktop/LibriSpeech/train-clean-100/5...,But the cold drove us out and making a large f...
998,998,/home/g7/Desktop/LibriSpeech/train-clean-100/1...,Only i beg it shall not be before midnight


In [4]:
df_specs = df_specs.sample(frac=1, random_state=42).reset_index(drop=True)
df_specs

,Unnamed: 0,audio,sentence
0,521,/home/g7/Desktop/LibriSpeech/train-clean-100/1...,Love and water brought back all her strength s...
1,737,/home/g7/Desktop/LibriSpeech/train-clean-100/1...,That the captain determined to run into wigwam...
2,740,/home/g7/Desktop/LibriSpeech/train-clean-100/4...,Growled sam kitteridge bitterly resenting the ...
3,660,/home/g7/Desktop/LibriSpeech/train-clean-100/7...,Sir sidney colvin regrets that the love letter...
4,411,/home/g7/Desktop/LibriSpeech/train-clean-100/4...,Leading their young full fledged and about as ...
...,...,...,...
995,106,/home/g7/Desktop/LibriSpeech/train-clean-100/4...,And unconscious of the danger stood her ground...
996,270,/home/g7/Desktop/LibriSpeech/train-clean-100/3...,And every knight shall have a squire and two y...
997,860,/home/g7/Desktop/LibriSpeech/train-clean-100/7...,His appearance was welcomed by a joyful shout ...
998,435,/home/g7/Desktop/LibriSpeech/train-clean-100/2...,Well isabel you must be aware that it is an aw...


In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER-uncased")
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER-uncased")
nlp = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

2024-10-06 19:41:38.031624: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-06 19:41:38.052583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 19:41:38.052607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 19:41:38.053175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 19:41:38.056931: I tensorflow/core/platform/cpu_feature_guar

In [6]:
import torch
from transformers import pipeline
from jiwer import wer

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

pipe = pipeline(
    "automatic-speech-recognition",
    model=finetuned_model,
    tokenizer=finetuned_processor.tokenizer,
    feature_extractor=finetuned_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

n = len(df_specs)
wer_accumulator = []
cer_accumulator = []
pred_entities = []
predictions = []
sentences = []

for i in range (n):
  sample = df_specs['audio'][i]
  prediction = pipe(sample)["text"]
  print("prediction: ", prediction)
  print("sentence: ", df_specs['sentence'][i])
  #list of predictions
  predictions.append(prediction)
  #list of true sentences
  sentences.append(df_specs['sentence'][i])
  #calculate WER
  wer_ans = wer(prediction.lower(),df_specs['sentence'][i].lower())
  wer_accumulator.append(wer_ans)

# Calculate WER
avg_wer = 100 * (sum(wer_accumulator) / len(wer_accumulator))

print(f"Average Word Error Rate: {avg_wer:.2f}%")

prediction:  Love and water brought back all her strength she got under the water and pulled and pulled with her whole might till at last she got one leg out the other easily followed how she got him into the boat she never could tell
sentence:  Love and water brought back all her strength she got under the water and pulled and pulled with her whole might till at last she got one leg out the other easily followed how she got him into the boat she never could tell
prediction:  That the captain determined to run into wigwam cove this is a snug little harbour not far from cape horn and here at christmas eve we anchored in smooth water the only thing which reminded us of the gale outside
sentence:  That the captain determined to run into wigwam cove this is a snug little harbour not far from cape horn and here at christmas eve we anchored in smooth water the only thing which reminded us of the gale outside
prediction:  Growled sam kitteridge bitterly resenting the stern edict which forbade

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


prediction:  And what a poor compensation it seemed to those who had taken part in extracting it as they stood gazing upon the remains of the villain and his victim the swarth ruffian dangling from the branch above
sentence:  And what a poor compensation it seemed to those who had taken part in exacting it as they stood gazing upon the remains of the villain and his victim the swarth ruffian dangling from the branch above
prediction:  If the government could not back up such a declaration the other party did not wish the matter tampered with as cheap labor was necessary for raising cotton sugar and other products in which the living of millions of people depended
sentence:  If the government could not back up such a declaration the other party did not wish the matter tampered with as cheap labor was necessary for raising cotton sugar and other products on which the living of millions of people depended
prediction:  Just wait a moment and i will give you something and she disappeared in

In [7]:
# Calculate NER
#align first 
import jiwer
predictions = [prediction.lower() for prediction in predictions]
sentences = [sentence.lower() for sentence in sentences]
out = jiwer.process_words(
    predictions,
    sentences,
)
visualization = jiwer.visualize_alignment(out)
ref_list = []
hyp_list = []

print("out:", out)
print("visualize: ",jiwer.visualize_alignment(out))
lines = visualization.split('\n')
for line in lines:
    if line.startswith("REF:"):
        ref_list.append(line.strip().replace("REF: ", ""))
    elif line.startswith("HYP:"):
        hyp_list.append(line.strip().replace("HYP: ", ""))

# Now you have REF and HYP lines in two separate lists
print("References:", ref_list)
print("Hypotheses:", hyp_list)


out: WordOutput(references=[['love', 'and', 'water', 'brought', 'back', 'all', 'her', 'strength', 'she', 'got', 'under', 'the', 'water', 'and', 'pulled', 'and', 'pulled', 'with', 'her', 'whole', 'might', 'till', 'at', 'last', 'she', 'got', 'one', 'leg', 'out', 'the', 'other', 'easily', 'followed', 'how', 'she', 'got', 'him', 'into', 'the', 'boat', 'she', 'never', 'could', 'tell'], ['that', 'the', 'captain', 'determined', 'to', 'run', 'into', 'wigwam', 'cove', 'this', 'is', 'a', 'snug', 'little', 'harbour', 'not', 'far', 'from', 'cape', 'horn', 'and', 'here', 'at', 'christmas', 'eve', 'we', 'anchored', 'in', 'smooth', 'water', 'the', 'only', 'thing', 'which', 'reminded', 'us', 'of', 'the', 'gale', 'outside'], ['growled', 'sam', 'kitteridge', 'bitterly', 'resenting', 'the', 'stern', 'edict', 'which', 'forbade', 'free', 'born', 'citizens', 'to', 'burn', 'as', 'much', 'gunpowder', 'as', 'they', 'liked', 'on', 'that', 'glorious', 'day', 'last', 'year', 'jimmy', 'got', 'his', 'arm', 'blown',

In [8]:
# calculate NER for each true sentence
true_entities = []
for i in range(len(ref_list)):
    result = nlp(ref_list[i])
    doc_entities = []
    for entity in result:
        # Normalize B-PER and I-PER to PER and include only if PER
        if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
            doc_entities.append((entity['word'], 'PER'))
    # if doc_entities:  # Only append if there are PER entities in the document
    true_entities.append(doc_entities)
print("len true_entity: ", len(true_entities))
print("true_entities: ", true_entities)

len true_entity:  577
true_entities:  [[('sydney', 'PER'), ('col', 'PER'), ('##vin', 'PER'), ('ke', 'PER'), ('##ats', 'PER'), ('fanny', 'PER')], [], [('isabel', 'PER')], [], [], [], [], [], [], [('fe', 'PER'), ('##ssel', 'PER'), ('ala', 'PER'), ('##po', 'PER'), ('salt', 'PER'), ('##are', 'PER'), ('##lo', 'PER')], [], [], [('cr', 'PER'), ('##eman', 'PER'), ('##te', 'PER')], [], [('to', 'PER'), ('crystal', 'PER')], [('diana', 'PER')], [('gu', 'PER'), ('##ene', 'PER'), ('##ver', 'PER')], [], [], [], [('ur', 'PER'), ('##o', 'PER')], [('pr', 'PER'), ('##udence', 'PER'), ('mar', 'PER'), ('##gar', 'PER'), ('##it', 'PER'), ('delicious', 'PER'), ('pr', 'PER'), ('##udence', 'PER')], [], [('os', 'PER'), ('carly', 'PER'), ('##le', 'PER'), ('barbara', 'PER'), ('ma', 'PER'), ('##mma', 'PER')], [('sc', 'PER'), ('##ag', 'PER'), ('##zi', 'PER')], [('eric', 'PER'), ('gunn', 'PER'), ('##b', 'PER'), ('##jo', 'PER'), ('##rn', 'PER')], [('henry', 'PER')], [], [], [], [], [], [], [], [('am', 'PER')], [('hi',

In [9]:
#merge names to one word if they are in the same entity depending on the hashtags #
true_entities_merged = []
for i in range(len(true_entities)):
    doc_entities = true_entities[i]
    doc_entities_merged = []
    for j in range(len(doc_entities)):
        if doc_entities[j][1] == 'PER':
            doc_entities_merged.append((doc_entities[j][0], 'PER'))
            while(j+1 < len(doc_entities) and doc_entities[j+1][1] == 'PER' and doc_entities[j+1][0][0] =='#'):
                #remove the hashtag and merge the names
                doc_entities_merged[-1] = (doc_entities_merged[-1][0] + doc_entities[j+1][0][2:], 'PER')
                doc_entities[j+1] = (doc_entities[j+1][0], '*')
                j += 1
            # doc_entities_merged.append(doc_entities[j])
    true_entities_merged.append(doc_entities_merged)

print("true_entities_merged: ", true_entities_merged)           

true_entities_merged:  [[('sydney', 'PER'), ('colvin', 'PER'), ('keats', 'PER'), ('fanny', 'PER')], [], [('isabel', 'PER')], [], [], [], [], [], [], [('fessel', 'PER'), ('alapo', 'PER'), ('saltarelo', 'PER')], [], [], [('cremante', 'PER')], [], [('to', 'PER'), ('crystal', 'PER')], [('diana', 'PER')], [('guenever', 'PER')], [], [], [], [('uro', 'PER')], [('prudence', 'PER'), ('margarit', 'PER'), ('delicious', 'PER'), ('prudence', 'PER')], [], [('os', 'PER'), ('carlyle', 'PER'), ('barbara', 'PER'), ('mamma', 'PER')], [('scagzi', 'PER')], [('eric', 'PER'), ('gunnbjorn', 'PER')], [('henry', 'PER')], [], [], [], [], [], [], [], [('am', 'PER')], [('hiaosinth', 'PER'), ('cornell', 'PER'), ('udo', 'PER')], [('d', 'PER'), ("'", 'PER'), ('artagnan', 'PER')], [], [('jane', 'PER')], [('nielson', 'PER'), ('billy', 'PER'), ('alice', 'PER'), ('greggory', 'PER')], [('myriam', 'PER'), ('nora', 'PER')], [('verry', 'PER'), ('father', 'PER'), ('cassie', 'PER'), ('grandfather', 'PER')], [], [], [('elizabet

In [10]:
#add the other words add (add entity * to it) to the true entities merged list to have the same length as the predictions
final_true_entities_merged = [] #list of lists of tuples (word, entity) for each sentence for all the words in the sentence
for i in range(len(true_entities_merged)): #for each sentence 
    doc_entities = true_entities_merged[i] 
    ref_sentence = ref_list[i].split()
    k = 0

    final_true_entities = []
    for j in range(len(ref_sentence)):
        if (len(doc_entities) == k or doc_entities[k][0].lower() not in ref_sentence[j].lower()): #if the word is not in the entities list add it with entity *
            # print(ref_sentence[j], doc_entities[k][0])
            final_true_entities.append((ref_sentence[j], '*'))
        else:
            #if the word is a substring of the entity word, add the original word to the list iinstead of the entity word (WITH THE ENTITY)
            final_true_entities.append((ref_sentence[j], doc_entities[k][1]))
            k += 1
    final_true_entities_merged.append(final_true_entities)
print("true_entities_merged: ", final_true_entities_merged)
    

true_entities_merged:  [[('sir', '*'), ('sydney', 'PER'), ('colvin', 'PER'), ('regrets', '*'), ('that', '*'), ('the', '*'), ('love', '*'), ('letters', '*'), ('of', '*'), ('keats', 'PER'), ('to', '*'), ('fanny', 'PER'), ('were', '*'), ('ever', '*'), ('published', '*'), ('it', '*'), ('would', '*'), ('be', '*'), ('as', '*'), ('reasonable', '*'), ('in', '*'), ('my', '*'), ('opinion', '*')], [('leading', '*'), ('their', '*'), ('young', '*'), ('full', '*'), ('fledged', '*'), ('and', '*'), ('about', '*'), ('as', '*'), ('large', '*'), ('and', '*'), ('strong', '*'), ('as', '*'), ('the', '*'), ('parents', '*'), ('squirrels', '*'), ('dry', '*'), ('and', '*'), ('elastic', '*'), ('after', '*'), ('the', '*'), ('storms', '*'), ('were', '*'), ('busy', '*'), ('about', '*'), ('their', '*'), ('stores', '*'), ('of', '*'), ('pine', '*'), ('nuts', '*'), ('and', '*'), ('the', '*'), ('latest', '*'), ('golden', '*'), ('rods', '*'), ('were', '*'), ('still', '*'), ('in', '*'), ('bloom', '*'), ('though', '*'), ('

In [11]:
for i in range(len(final_true_entities_merged)):
    ref_sentence = ref_list[i].split()
    doc_entities = final_true_entities_merged[i]

    for j in range(len(ref_sentence)):
        if (len(ref_sentence[j]) != len(doc_entities[j][0])):
            print("ref_sentence: ", ref_sentence[j])
            print("doc_entities: ", doc_entities[j][0])

In [12]:
# calculate NER for each prediction
pred_entities = []
for i in range(len(hyp_list)):
    result = nlp(hyp_list[i])
    doc_entities = []
    for entity in result:
        # Normalize B-PER and I-PER to PER and include only if PER
        if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
            doc_entities.append((entity['word'], 'PER'))
        else:
            doc_entities.append((entity['word'], '*'))
    # if doc_entities:  # Only append if there are PER entities in the document
    pred_entities.append(doc_entities)
print("len pred_entities: ", len(pred_entities))
print("pred_entities: ", pred_entities)

len pred_entities:  577
pred_entities:  [[('sidney', 'PER'), ('col', 'PER'), ('##vin', 'PER'), ('ke', 'PER'), ('##ats', 'PER'), ('fanny', 'PER')], [], [('paris', '*'), ('isabel', 'PER')], [('mari', '*')], [], [], [('ry', 'PER'), ('##nch', 'PER')], [], [], [('trojan', '*'), ('fe', '*'), ('##e', '*'), ('rome', '*'), ('lap', '*'), ('##o', '*'), ('salt', '*'), ('##ere', '*'), ('##llo', '*')], [], [], [('miss', 'PER'), ('gram', 'PER'), ('##mont', 'PER')], [], [('##r', '*'), ('old', 'PER'), ('toll', 'PER'), ('cr', 'PER'), ('##iste', 'PER'), ('##l', 'PER')], [('diana', 'PER')], [('gu', 'PER'), ('##ene', 'PER'), ('##ver', 'PER'), ('table', '*'), ('westminster', '*')], [], [], [], [('ud', '*'), ('##o', '*')], [('pr', 'PER'), ('##udence', 'PER'), ('marguerite', 'PER'), ('delicious', 'PER'), ('pr', 'PER'), ('##udence', 'PER')], [], [('carly', 'PER'), ('##le', 'PER'), ('barbara', 'PER'), ('ma', 'PER'), ('##mma', 'PER')], [('ska', 'PER'), ('##ggs', 'PER'), ('##y', 'PER')], [('eric', 'PER'), ('gunn'

In [13]:
#merge names to one word if they are in the same entity depending on the hashtags #
pred_entities_merged = []
for i in range(len(pred_entities)):
    doc_entities = pred_entities[i]
    doc_entities_merged = []
    for j in range(len(doc_entities)):
        if doc_entities[j][1] == 'PER':
            doc_entities_merged.append((doc_entities[j][0], 'PER'))
            while(j+1 < len(doc_entities) and doc_entities[j+1][1] == 'PER' and doc_entities[j+1][0][0] =='#'):
                #remove the hashtag and merge the names
                doc_entities_merged[-1] = (doc_entities_merged[-1][0] + doc_entities[j+1][0][2:], 'PER')
                doc_entities[j+1] = (doc_entities[j+1][0], '*')
                j += 1
            # doc_entities_merged.append(doc_entities[j])
    pred_entities_merged.append(doc_entities_merged)

print("pred_entities_merged: ", pred_entities_merged)

pred_entities_merged:  [[('sidney', 'PER'), ('colvin', 'PER'), ('keats', 'PER'), ('fanny', 'PER')], [], [('isabel', 'PER')], [], [], [], [('rynch', 'PER')], [], [], [], [], [], [('miss', 'PER'), ('grammont', 'PER')], [], [('old', 'PER'), ('toll', 'PER'), ('cristel', 'PER')], [('diana', 'PER')], [('guenever', 'PER')], [], [], [], [], [('prudence', 'PER'), ('marguerite', 'PER'), ('delicious', 'PER'), ('prudence', 'PER')], [], [('carlyle', 'PER'), ('barbara', 'PER'), ('mamma', 'PER')], [('skaggsy', 'PER')], [('eric', 'PER'), ('gunnbiorn', 'PER')], [('henry', 'PER')], [], [], [], [], [], [], [], [], [('hyacinth', 'PER'), ('coronel', 'PER'), ('udo', 'PER')], [('d', 'PER'), ("'", 'PER'), ('artagnan', 'PER')], [], [('jane', 'PER')], [('neilson', 'PER'), ('billy', 'PER'), ('alice', 'PER'), ('greggory', 'PER')], [('miriam', 'PER'), ('nora', 'PER')], [('verry', 'PER'), ('cassy', 'PER'), ('grandfather', 'PER')], [], [], [('elizabeth', 'PER')], [('hetty', 'PER')], [], [('agatha', 'PER')], [], [], 

In [14]:
#add the other words add (add entity * to it) to the true entities merged list to have the same length as the predictions
final_pred_entities_merged = [] #list of lists of tuples (word, entity) for each sentence for all the words in the sentence
for i in range(len(pred_entities_merged)): #for each sentence 
    doc_entities = pred_entities_merged[i] 
    hyp_sentence = hyp_list[i].split()
    k = 0

    final_pred_entities = []
    for j in range(len(hyp_sentence)):
        if (len(doc_entities) == k or doc_entities[k][0].lower() not in hyp_sentence[j].lower()): #if the word is not in the entities list add it with entity *
            # print(ref_sentence[j], doc_entities[k][0])
            final_pred_entities.append((hyp_sentence[j], '*'))
        else:
            #if the word is a substring of the entity word, add the original word to the list iinstead of the entity word (WITH THE ENTITY)
            final_pred_entities.append((hyp_sentence[j], doc_entities[k][1]))
            k += 1
    final_pred_entities_merged.append(final_pred_entities)
print("final_pred_entities_merged: ", final_pred_entities_merged)

final_pred_entities_merged:  [[('sir', '*'), ('sidney', 'PER'), ('colvin', 'PER'), ('regrets', '*'), ('that', '*'), ('the', '*'), ('love', '*'), ('letters', '*'), ('of', '*'), ('keats', 'PER'), ('to', '*'), ('fanny', 'PER'), ('were', '*'), ('ever', '*'), ('published', '*'), ('it', '*'), ('would', '*'), ('be', '*'), ('as', '*'), ('reasonable', '*'), ('in', '*'), ('my', '*'), ('opinion', '*')], [('leading', '*'), ('their', '*'), ('young', '*'), ('full', '*'), ('fledged', '*'), ('and', '*'), ('about', '*'), ('as', '*'), ('large', '*'), ('and', '*'), ('strong', '*'), ('as', '*'), ('the', '*'), ('parents', '*'), ('squirrels', '*'), ('dry', '*'), ('and', '*'), ('elastic', '*'), ('after', '*'), ('the', '*'), ('storms', '*'), ('were', '*'), ('busy', '*'), ('about', '*'), ('their', '*'), ('stores', '*'), ('of', '*'), ('pine', '*'), ('nuts', '*'), ('and', '*'), ('the', '*'), ('latest', '*'), ('goldenrods', '*'), ('****', '*'), ('were', '*'), ('still', '*'), ('in', '*'), ('bloom', '*'), ('though'

In [15]:
# Function to extract only the tags and replace '*' with 'O'
def extract_tags(nested_list):
    tags = []
    for sublist in nested_list:
        # Collect tags from each tuple in the sublist, replace '*' with 'O'
        sublist_tags = ['O' if tag == '*' else 'I-' + tag for word, tag in sublist]
        if sublist_tags:  # Only add non-empty lists
            tags.append(sublist_tags)
    return tags


In [16]:
true_extracted_tags = extract_tags(final_true_entities_merged)
predicted_extracted_tags = extract_tags(final_pred_entities_merged)

In [17]:
#calculate precision, recall, f1 score
from seqeval.metrics import precision_score, recall_score, f1_score

precision = precision_score(true_extracted_tags, predicted_extracted_tags)
recall = recall_score(true_extracted_tags, predicted_extracted_tags)
f1 = f1_score(true_extracted_tags, predicted_extracted_tags)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)


Precision:  0.7861111111111111
Recall:  0.771117166212534
F1:  0.7785419532324621
